In [1]:
import sys
import numpy as np
import pandas as pd
import seaborn as sns

#np.set_printoptions(threshold=sys.maxsize)

Import data:

In [2]:
data = pd.read_csv('weatherHistory.csv')

In [3]:
print(data)

                      Formatted Date        Summary Precip Type  \
0      2006-04-01 00:00:00.000 +0200  Partly Cloudy        rain   
1      2006-04-01 01:00:00.000 +0200  Partly Cloudy        rain   
2      2006-04-01 02:00:00.000 +0200  Mostly Cloudy        rain   
3      2006-04-01 03:00:00.000 +0200  Partly Cloudy        rain   
4      2006-04-01 04:00:00.000 +0200  Mostly Cloudy        rain   
5      2006-04-01 05:00:00.000 +0200  Partly Cloudy        rain   
6      2006-04-01 06:00:00.000 +0200  Partly Cloudy        rain   
7      2006-04-01 07:00:00.000 +0200  Partly Cloudy        rain   
8      2006-04-01 08:00:00.000 +0200  Partly Cloudy        rain   
9      2006-04-01 09:00:00.000 +0200  Partly Cloudy        rain   
10     2006-04-01 10:00:00.000 +0200  Partly Cloudy        rain   
11     2006-04-01 11:00:00.000 +0200  Partly Cloudy        rain   
12     2006-04-01 12:00:00.000 +0200  Partly Cloudy        rain   
13     2006-04-01 13:00:00.000 +0200  Partly Cloudy        rai

In [4]:
print(data.dtypes)

Formatted Date               object
Summary                      object
Precip Type                  object
Temperature (C)             float64
Apparent Temperature (C)    float64
Humidity                    float64
Wind Speed (km/h)           float64
Wind Bearing (degrees)      float64
Visibility (km)             float64
Loud Cover                  float64
Pressure (millibars)        float64
Daily Summary                object
dtype: object


Extract day, month, year from formatted date.

In [5]:
data['Date'] = pd.to_datetime(data['Formatted Date'], errors='ignore', utc=True)

In [6]:
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Day'] = data['Date'].dt.day
data['Hour'] = data['Date'].dt.hour

In [7]:
data.head()

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary,Date,Year,Month,Day,Hour
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.,2006-03-31 22:00:00+00:00,2006,3,31,22
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.,2006-03-31 23:00:00+00:00,2006,3,31,23
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.,2006-04-01 00:00:00+00:00,2006,4,1,0
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.,2006-04-01 01:00:00+00:00,2006,4,1,1
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.,2006-04-01 02:00:00+00:00,2006,4,1,2


In [8]:
summary = []
daily_summary = []
for index, row in data.iterrows():
    if row['Summary'] not in summary:
        summary.append(row['Summary'])
    if row['Daily Summary'] not in daily_summary:
        daily_summary.append(row['Daily Summary'])

In [9]:
print(summary)
print(daily_summary)
print('Summary has', len(summary), 'elements.\nDaily Summary has', len(daily_summary), 'elements.')

['Partly Cloudy', 'Mostly Cloudy', 'Overcast', 'Foggy', 'Breezy and Mostly Cloudy', 'Clear', 'Breezy and Partly Cloudy', 'Breezy and Overcast', 'Humid and Mostly Cloudy', 'Humid and Partly Cloudy', 'Windy and Foggy', 'Windy and Overcast', 'Breezy and Foggy', 'Windy and Partly Cloudy', 'Breezy', 'Dry and Partly Cloudy', 'Windy and Mostly Cloudy', 'Dangerously Windy and Partly Cloudy', 'Dry', 'Windy', 'Humid and Overcast', 'Light Rain', 'Drizzle', 'Windy and Dry', 'Dry and Mostly Cloudy', 'Breezy and Dry', 'Rain']
['Partly cloudy throughout the day.', 'Mostly cloudy throughout the day.', 'Foggy in the evening.', 'Foggy overnight and breezy in the morning.', 'Overcast throughout the day.', 'Partly cloudy until night.', 'Mostly cloudy until night.', 'Foggy starting overnight continuing until morning.', 'Foggy in the morning.', 'Partly cloudy until evening.', 'Partly cloudy starting in the morning.', 'Mostly cloudy starting overnight continuing until night.', 'Mostly cloudy until evening.',

In [10]:
# Plot the correlation between columns
#f, ax = plt.subplots(figsize=(18, 18))
#sns.heatmap(data.corr(), annot=True, linewidths=5, fmt='.1f', ax=ax)

Check if there is any missing value and if there is find out in which column.

In [11]:
data.isnull().any()

Formatted Date              False
Summary                     False
Precip Type                  True
Temperature (C)             False
Apparent Temperature (C)    False
Humidity                    False
Wind Speed (km/h)           False
Wind Bearing (degrees)      False
Visibility (km)             False
Loud Cover                  False
Pressure (millibars)        False
Daily Summary               False
Date                        False
Year                        False
Month                       False
Day                         False
Hour                        False
dtype: bool

Clear missing values

In [12]:
filtered_data = data.dropna()
filtered_data.isnull().any()

Formatted Date              False
Summary                     False
Precip Type                 False
Temperature (C)             False
Apparent Temperature (C)    False
Humidity                    False
Wind Speed (km/h)           False
Wind Bearing (degrees)      False
Visibility (km)             False
Loud Cover                  False
Pressure (millibars)        False
Daily Summary               False
Date                        False
Year                        False
Month                       False
Day                         False
Hour                        False
dtype: bool

Convert categorical labels to numerical labels.

In [13]:
from sklearn.preprocessing import LabelEncoder

labelEncoder = LabelEncoder()
data['Precip Type'] = labelEncoder.fit_transform(data['Precip Type'].fillna('-1'))

In [14]:
print(data['Precip Type'])

0        1
1        1
2        1
3        1
4        1
5        1
6        1
7        1
8        1
9        1
10       1
11       1
12       1
13       1
14       1
15       1
16       1
17       1
18       1
19       1
20       1
21       1
22       1
23       1
24       1
25       1
26       1
27       1
28       1
29       1
        ..
96423    1
96424    1
96425    1
96426    1
96427    1
96428    1
96429    1
96430    1
96431    1
96432    1
96433    1
96434    1
96435    1
96436    1
96437    1
96438    1
96439    1
96440    1
96441    1
96442    1
96443    1
96444    1
96445    1
96446    1
96447    1
96448    1
96449    1
96450    1
96451    1
96452    1
Name: Precip Type, Length: 96453, dtype: int32


Import scikit-learn

In [15]:
from sklearn.model_selection import train_test_split

Split data into train and test.

In [33]:
# y is the label
# X is the independent variables
X = data.iloc[:, 3:11]
y = data.iloc[:, 2]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)

print("X_train\n----------------------------------------------------------------------------------\n",X_train, "\n")
print("X_test\n----------------------------------------------------------------------------------\n",X_test, "\n")
print("y_train\n----------------------------------------------------------------------------------\n",y_train, "\n")
print("y_test\n----------------------------------------------------------------------------------\n",y_test, "\n")

X_train
----------------------------------------------------------------------------------
        Temperature (C)  Apparent Temperature (C)  Humidity  Wind Speed (km/h)  \
93676        12.805556                 12.805556      0.90            19.1268   
44598        13.955556                 13.955556      0.82            11.3666   
2264         -1.038889                 -3.933333      0.99             7.9695   
67116        10.938889                 10.938889      0.37            24.2788   
63582         2.227778                 -0.883333      0.79            11.0607   
5105          1.300000                 -2.605556      0.74            13.9587   
82128         5.600000                  3.844444      0.96             8.0500   
34991        13.422222                 13.422222      0.68            10.9641   
24742        13.305556                 13.305556      0.78            10.5777   
4353         23.794444                 23.794444      0.70             2.5438   
31237        23.8

In [17]:
####################################################################
#############              DECISION TREE           #################
####################################################################

In [36]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score

classifier = DecisionTreeClassifier(random_state=0)
classifier.fit(X_train, y_train)

cross_val_score(classifier, X_train, y_train, cv=10)

array([0.99319307, 0.9947401 , 0.9948948 , 0.99473847, 0.99303621,
       0.99458372, 0.99442897, 0.99288036, 0.99102306, 0.99303513])

In [37]:
from sklearn import tree
import graphviz

dot_data = tree.export_graphviz(classifier, out_file=None)
graph = graphviz.Source(dot_data)
graph.render('weather')

'weather.pdf'

In [20]:
####################################################################
#############              RANDOM FOREST           #################
####################################################################

In [38]:
from sklearn.ensemble import RandomForestClassifier
rf_classifier = RandomForestClassifier(n_estimators=100, max_depth=2, random_state=0)
rf_classifier.fit(X_train, y_train)

print(rf_classifier.feature_importances_)

[0.40238099 0.29958313 0.06455624 0.01304293 0.00238096 0.08896521
 0.         0.12909055]


Make prediction using classifier.

In [46]:
classified_data = rf_classifier.predict(X_test)
real_data = y_test

true_count = 0
for i in range(len(classified_data)):
    if classified_data[i] == real_data.iloc[i]:
        true_count += 1
print(len(classified_data), '|', true_count)
print('True percentage', true_count / len(classified_data))

31830 | 31426
True percentage 0.9873075714734527


In [24]:
####################################################################
#############                ADA BOOST             #################
####################################################################

In [48]:
from sklearn.ensemble import AdaBoostClassifier

ada_boost_classifier = AdaBoostClassifier(n_estimators=100)
scores = cross_val_score(ada_boost_classifier, X_train, y_train, cv=5)
scores.mean()

0.9403830773979983

In [29]:
####################################################################
#############         GRADIENT TREE BOOSTING       #################
####################################################################

In [50]:
from sklearn.ensemble import GradientBoostingClassifier

gradient_boost_classifier = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0, max_depth=1, random_state=0)
gradient_boost_classifier.fit(X_train, y_train)
gradient_boost_classifier.score(X_test, y_test)

0.9958215519949732

In [52]:
#TODO: 'Play with the parameters of the functions to get better result and convert data to log format and try loading data into database.' 